In [ ]:
import sympy
from sympy import Matrix, S, Symbol, symbols, I, zeros, eye
from sympy import simplify, expand, expand_complex, latex
import numpy as np
from IPython.display import Latex, display
import scipy
import scipy.linalg
import pandas as pd
from sympy import matrix2numpy
from google.colab import files

# Практическое занятие 17 Демченко Георгий Павлович
# Компьютерный практикум по алгебре на Python
## Матричные разложения: Холецкого, LDL, LU, QR. Жорданова форма.

### Задание 1.
Построить разложение Холецкого матриц
$$
M1=\left(
\begin{matrix}
1&-3&0\\
-3&-2&10\\
0&10&7
\end{matrix}
\right)
\quad
M2=\left(
\begin{matrix}
18&1 - 2I& -2\\
1 + 2I&4&-3I\\
-2&3I&5
\end{matrix}
\right)
$$
Проверить положительную определенность эрмитовой матрицы.

In [ ]:
# @title Функция для проверки существования разложения Холецкого и его нахождения
def CheckCholesky(matrix, needToCheckErmit = False):
  if (needToCheckErmit and not matrix.is_positive_definite):
    display(Latex(fr"\text{{Эрмитова матрица не имеет положительной определенности, разложение Холецкого невозможно}}"))
    return (None, None)
  if (not matrix.is_symmetric() and not needToCheckErmit):
    display(Latex(fr"\text{{Матрица не является симметричной, разложение Холецкого невозможно.}}"))
    return (None, None)

  L_mx = sympy.simplify(matrix.cholesky(hermitian = needToCheckErmit))
  Cholesky_decomposition = sympy.simplify(L_mx * L_mx.H) if  needToCheckErmit else sympy.simplify(L_mx * L_mx.T)
  display(*[Latex(fr"L \  = \  {latex(L_mx)}"), Latex(fr"L \cdot L^{'H' if needToCheckErmit else 'T'} \ = {latex(Cholesky_decomposition)}")])
  return (L_mx, Cholesky_decomposition)

In [ ]:
M_1 = Matrix([[1, -3, 0], [-3, -2, 10], [0, 10, 7]])
M_2 = Matrix([[18, 1 - 2 * I, -2], [1 + 2 * I, 4, -3 * I], [-2, 3 * I, 5]])

display(Latex(fr"\text{{[1].}} M_{1}"))
L_m1, Check_m1 = CheckCholesky(M_1)
display(Latex(fr"\text{{[2].}} M_{2}"))
L_m2, Check_m2 = CheckCholesky(M_2, True)

### Задание 2.
Построить  LDL разложение для матриц Задания 1.

In [ ]:
# @title Функция для проверки существования LDL разложения и его нахождения
def LDLDecomposition(matrix, ermitMx = False):
  if (ermitMx and not matrix.is_positive_definite):
    display(Latex(fr"\text{{Эрмитова матрица не имеет положительной определенности, LDL разложение невозможно}}"))
    return (None, None, None)
  if (not matrix.is_symmetric() and not ermitMx):
    display(Latex(fr"\text{{Матрица не является симметричной, LDL разложение невозможно.}}"))
    return (None, None, None)
  L_mx, D_mx = [sympy.simplify(mx) for mx in matrix.LDLdecomposition(hermitian = ermitMx)]
  LDL_decomposition = sympy.simplify(L_mx * D_mx * L_mx.H) if ermitMx else sympy.simplify(L_mx * D_mx * L_mx.T)
  display(*[Latex(fr"L \  = \  {latex(L_mx)}"), Latex(fr"D \ =  \ {latex(D_mx)}"), Latex(fr"L \cdot D \cdot L^{'H' if ermitMx else 'T'} \ = {latex(LDL_decomposition)}")])
  return (L_mx, D_mx, LDL_decomposition)

In [ ]:
display(Latex(fr"\text{{[1].}} M_{1}"))
L_m1, D_m1, Check_m1 = LDLDecomposition(M_1)
display(Latex(fr"\text{{[2].}} M_{2}"))
L_m2, D_m2, Check_m2 = LDLDecomposition(M_2 , True)

### Задание  3.
Построить  LU разложение для матрицы
$$
V=\left(
\begin{matrix}
5&-2 - I&3 - 4I&1 + 4I\\
1 - I&-2&5 - I&2 - I\\
5&6 + I&0&5
\end{matrix}
\right)
$$

In [ ]:
# @title Функция для нахождения LU разложения произвольой матрицы
def LUDecomposition(matrix):
  LU_decomposition = None
  L_mx, U_mx = [sympy.simplify(expand(mx)) for mx in matrix.LUdecomposition()[:2]]
  perm_lst = matrix.LUdecomposition()[-1];
  if (len(perm_lst) == 0):
    LU_decomposition = L_mx * U_mx
    # return (L_mx, U_mx, LU_decomposition)
  no_rows, no_columns = matrix.shape
  LU_decomposition = simplify(expand((L_mx * U_mx).permuteBkwd(perm_lst)))
  display(*[Latex(fr"L \  = \  {latex(L_mx)}"), Latex(fr"U \ =  \ {latex(U_mx)}"), Latex(fr"\text{{permutations =}} {latex(perm_lst)}"),Latex(fr"L \cdot U {'with permutations' if len(perm_lst) != 0 else ''} \ = {latex(LU_decomposition)}")])
  return (L_mx, U_mx, LU_decomposition)

In [ ]:
V_mx = Matrix([[5, -2 - I, 3 - 4 * I, 1 + 4 * I],
               [1 - I, -2, 5 - I, 2 - I],
               [5, 6 + I, 0 , 5]])

L_v, U_v, Check_v = LUDecomposition(V_mx)

### Задание  4.
Построить  QR разложение для матрицы
$$
A=\left(
\begin{matrix}
3 + i&  2 & -i & 4 - 2i\\
-2 & -3 &  i & -3 + i\\
1 + i & -1 &  0 & 1 - i\\
-1 + i &  -4 & i & -2
\end{matrix}
\right)
$$
показать, что $A = QR$.

In [ ]:
# @title Функция для нахождения QR разложения матрицы
def QRDecompositionSympy(matrix, needSimplify = False):
  if (needSimplify):
    Q_mx, R_mx = [simplify(expand(mx)) for mx in matrix.QRdecomposition()]
    QR_decomposition = sympy.simplify(expand(Q_mx * R_mx))
  else:
    Q_mx, R_mx = matrix.QRdecomposition()
    QR_decomposition = simplify(Q_mx * R_mx)
  display(*[Latex(fr"Q \  = \  {latex(Q_mx)}"), Latex(fr"R \ =  \ {latex(R_mx)}"), Latex(fr"Q \cdot R = {latex(QR_decomposition)}")])
  return (Q_mx, R_mx, QR_decomposition)


def QRDecompositionNumPy(matrix):
  Q_mx, R_mx = scipy.linalg.qr(matrix)
  QR_decomposition = Q_mx @ R_mx
  display(Latex("""Q = {},\\\\ R = {}""".format(*map(latex, map(Matrix, (Q_mx, R_mx))))))
  return (Q_mx, R_mx, QR_decomposition)

In [ ]:
# При попытке упроситить числа в матрице с использованием simplify и expand получаются NaN, которые невозможно преобразовать к числам и приближенно посчитать
A_mx = Matrix([[3 + I, 2, -I, 4 - 2 * I],
            [-2, -3, I, -3 + I],
            [1 + I, - 1, 0, 1 - I],
            [-1 + I, -4, I, -2]])

A_mx_np = np.array([[3 + 1j, 2, -1j, 4 - 2 * 1j],
                    [-2, -3, 1j, -3 + 1j],
                    [1 + 1j, -1, 0, 1 - 1j],
                    [-1 + 1j, -4, 1j, -2]])

display(Latex(fr"\text{{[1]. Применяя QR разложение через sympy:}}"))
Q_a, R_a, Check_a = QRDecompositionSympy(A_mx)
display(Latex(fr"\text{{[2]. Применяя QR разложение через numpy:}}"))
Q_a_2, R_a_2, Check_a_2 = QRDecompositionNumPy(A_mx_np)

### Задание  5.
Найти SVD c sympy и numpy
$$
\left(
\begin{matrix}
1 & 0 & 0 & -2\\
0 & 1 & 0 & 1\\
0 & 0 & 3 & 1
\end{matrix}
\right)
$$

In [ ]:
# Будем находить через sympy
Mx_5 = Matrix([[1, 0 , 0, -2], [0 , 1, 0, 1], [0, 0, 3, 1]])

Mx_pow_2 = Mx_5.T * Mx_5
lstEnums = [Mx_pow_2.eigenvects()[i][0] for i in range(len(Mx_pow_2.eigenvects()))]
lstEVecs = [Mx_pow_2.eigenvects()[i][2][j] for i in range(len(Mx_pow_2.eigenvects())) for j in range(len(Mx_pow_2.eigenvects()[i][2]))]
lst_norm_vec = [vector.normalized() for vector in lstEVecs]
e0, e11, e12, e4 = lst_norm_vec

e120 = e12
e12 = (e12 - e11.dot(e12) * e11).normalized()
P = e4.row_join(e11).row_join(e12).row_join(e0)
e11, e12 = sympy.matrices.dense.GramSchmidt([e11, e120], orthonormal=True)

sigma = (2, 1, 1)
f1, f2, f3 = [Mx_5 * ei / sigma[i] for i, ei in enumerate((e4, e11, e12))]
Q = f1.row_join(f2).row_join(f3)
Sig = sympy.Matrix([[2, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
display(Latex('Q = {}, Sig = {},P = {}, Q  Sig  P^T = {}'.format(*map(latex, (Q, Sig, P, Q * Sig * P.T)))))

## Задача 6*
Считать из файла "matrix_task_6.xlsx" матрицу $A$, построить для нее разложение lu, ldl и qr. Все полученные матрицы записать в файл "matrix_task_6_ans.xlsx" на отдельные листы

In [ ]:
uploaded_files_dct = files.upload()
file_name = [name for name in uploaded_files_dct.keys()][0]
matrix_df = pd.read_excel(file_name, sheet_name = "Sheet1", header = None)
matrix_sympy = sympy.Matrix(matrix_df.to_numpy())

In [ ]:
# Заметим, что числа матрицы в эксель таблице заданы произвольно в промежутке от -10 до 10 :D, поэтому при каждом новом скачивании будет получается произвольная матрица
display(Latex(fr"\text{{[0]. Исходная матрица:}} {latex(matrix_sympy)}"))
display(Latex(fr"\text{{[1]. LU разложение:}}"))
L_ind, U_ind, mx_check = LUDecomposition(matrix_sympy)
display(Latex(fr"\text{{[2]. LDL разложение:}}"))
L_ind_2, D_ind, mx_check_2 = LDLDecomposition(matrix_sympy)
display(Latex(fr"\text{{[3]. QR разложение:}}"))
Q_ind, R_ind, mx_check_3 = QRDecompositionSympy(matrix_sympy, True)

lst_mx_data = [(L_ind, "L"), (U_ind, "U"), (mx_check, "Matrix check LU"), (Q_ind, "Q"), (R_ind, "R"), (mx_check_3, "Matrix check QR")]

file_name_load = "matrix_task_6_ans.xlsx"
with pd.ExcelWriter(file_name_load, mode = "w") as writer_w:
  matrix_df.to_excel(writer_w, sheet_name = "Matrix", header = False, index = False)

with pd.ExcelWriter(file_name_load, mode = "a") as writer_2:
  for i in range(len(lst_mx_data)):
    cur_mx, cur_name = lst_mx_data[i]
    cur_np_array = np.array(cur_mx.tolist()).astype(np.float64)
    cur_df = pd.DataFrame(data = cur_np_array)
    cur_df.to_excel(writer_2, sheet_name = cur_name, header = False, index = False)

files.download(file_name_load)